In [1]:
import numpy as np

from weight import *
from utils import save, load
from library import *
from process_library_terms import *

In [ ]:
%%prun # profiling

### load data
# vicsek:
#arrays.npy - normal. arrays2.npy - very few particles. arrays3.npy - high density. arrays4.npy - high density, low noise.
#arrays5.npy - small domain, one clump
#datasets = ['arrays.npy', 'arrays2.npy', 'arrays3.npy']#, 'arrays5.npy']
#datasets = ['data_bu.npy'] # burgers' equation data
## it may be interesting to reintroduce regression on multiple datasets,
## but for now we're just not going to do it

dataset = 'data_bu.npy' 
#for i, dset in enumerate(datasets):
positions, vs, deltat, dims = load(dataset, 4)
world_size = np.array([dims[0], dims[0], dims[1]])
#world_size[2] = 50 # restrict to first 50 time steps
data_dict = {}
data_dict['v'] = vs
v_obs = Observable('v', 1)
observables = [v_obs]

# fix random seed
np.random.seed(1)

# initial setup of dataset
srd = SRDataset(world_size, data_dict, positions, observables, 
                kernel_sigma=2, cg_res=4, deltat=deltat, cutoff=8)
#kernel_sigma=1 or smaller

# initialize libraries, domains, and weights
srd.make_libraries(max_complexity=4, max_observables=3)
dom_width = 10
dom_time = 20
srd.make_domains(ndomains=50, domain_size=[dom_width, dom_width, dom_time], pad=0)
srd.make_weights(m=6, qmax=0)
srd.make_library_matrices()

save('Q_bu.npy', srd.dxs, srd.libs)

In [2]:
_, libs = load('Q_bu.npy', 2)
libs = libs.item()

In [3]:
### TO DO: MAKE THIS WORK
from identify_models import *
opts = dict()
opts['threshold'] = "'threshold'"
opts['gamma'] = 2
opts['verbose'] = False
opts['delta'] = 1e-8 # apparently this reaches machine precision as the relative error
opts['char_sizes'] = libs[0].col_weights
opts1 = opts.copy()
opts1['char_sizes'] = libs[1].col_weights
#eqs0, lambdas0, derived_eqs0, excluded_terms = identify_equations(Q0, opts, terms0, observables, threshold=1e-5, 
#                       max_complexity=None, max_equations=999, timed=True)
#print(excluded_terms)
#eqs1, lambdas1, derived_eqs1, excluded_terms = identify_equations(Q1, opts1, terms1, observables, threshold=1e-5, 
#                       max_complexity=None, max_equations=999, timed=True, excluded_terms=excluded_terms)
#print(excluded_terms)
eqs, lambdas, derived_eqs, excluded_terms = interleave_identify([libs[0].Q, libs[1].Q], 
[opts, opts1], [libs[0].terms, libs[1].terms], threshold=1e-4)

[0.01 s]
Identified model: 0.6406829298362364 * dt dj rho[v_j] + 1.0 * dt^2 rho = 0 (order 4, residual 4.37e-05)
[0.04 s]
Identified model: 1.0 * dt^3 rho = 0 (order 5, residual 1.24e-05)
[0.05 s]
Identified model: 1.0 * rho * dt rho[v_i] + -0.9999999999999996 * rho * rho[v_i] = 0 (order 5, residual 4.10e-12)


In [ ]:
## not planning to use most of this code

from ipynb.fs.full.sparse_reg import *
opts = dict()
opts['threshold'] = "'pareto'"
#opts['verbose'] = True
#opts['brute_force'] = False
#opts['threshold'] = "'threshold'"
#opts['n_terms'] = len(terms0)
#opts['gamma'] = 1.2
opts['char_sizes'] = srd.libs[0].col_weights
Xi, lambd, best_term, lambda1 = sparse_reg(srd.libs[0].Q, opts=opts)
print(srd.libs[0].Q.shape)

In [8]:
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {srd.libs[0].terms[i]}")
print("Model residual:", lambd)
print("One-term model residual:", lambda1)
print(f"Best term (#{best_term}):", srd.libs[0].terms[best_term])

[Term 19] 1.0 * rho[v_j] * rho[v_j]
Model residual: 0.0030944418121483697
One-term model residual: 1.294046607514735e-05
Best term (#6): dt^3 rho


In [5]:
### not fixed
opts['row_norms'] = row_weights0
Xip, lambdp, best_termp, lambda1p = sparse_reg(Q0, *opts)
for i in range(len(Xip)):
    if Xip[i]!=0:
        print(f"[Term {i}] {Xip[i]} * {terms0[i]}")
print("Model residual:", lambdp)
print("One-term model residual:", lambda1p)
print(f"Best term (#{best_termp}):", terms0[best_termp])

[Term 12] 1.0 * dt v_j * dt v_j
[Term 15] -0.0036532437302029016 * dj v_k * v_j * v_k
Model residual: 9.320023473785616e-06
One-term model residual: 2.4963892321884583e-05
Best term (#12): dt v_j * dt v_j


In [27]:
#opts['verbose'] = True
opts['char_sizes'] = srd.libs[1].col_weights
opts['avoid'] = []
opts['row_norms'] = None
Xi, lambd, best_term, lambda1 = sparse_reg(srd.libs[1].Q, opts=opts)
print(srd.libs[1].Q.shape)

(100, 18)


In [25]:
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {srd.libs[1].terms[i]}")
print("Model residual:", lambd)
print("One-term model residual:", lambda1)
print("Best term:", srd.libs[1].terms[best_term])

[Term 13] 1.0 * rho * dt rho[v_i]
Model residual: 0.17967234316108288
One-term model residual: 0.00010415619811026539
Best term: dt^2 rho[v_i]


In [6]:
### not fixed
for term, size in zip(terms1, char_sizes1):
    print(term, size)

NameError: name 'terms0' is not defined

In [23]:
continuity_terms = ['dt rho', 'dj rho[v_j]']
#continuity_terms = ['dt rho * rho', 'rho * rho * dj v_j', 'dj rho * rho * v_j']
#continuity_terms = ['dt rho', 'dj rv_j']
col_numbers = [find_term(srd.libs[0].terms, name) for name in continuity_terms]
Xi, lambd = regress(srd.libs[0].Q, col_numbers)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {srd.libs[0].terms[i]}.")# (Char. size: {srd.libs[0].col_weights[i]})")
print("Model residual:", lambd)

[Term 3] 1.0 * dt rho.
[Term 14] 0.6228749486744181 * dj rho[v_j].
Model residual: 0.0003346546143637365


In [22]:
burger_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]']
#burger_terms = ['rho * dt rv_i', 'dj rv_i * rv_j']
col_numbers = [find_term(srd.libs[1].terms, name) for name in burger_terms]
Xi, lambd = regress(srd.libs[1].Q, col_numbers)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {srd.libs[1].terms[i]}. (Char. size: {srd.libs[1].col_weights[i]})")
print("Model residual:", lambd)

[Term 10] 1.0 * dt rho[v_i]. (Char. size: 0.4059066427962728)
[Term 16] 0.6359338318692195 * dj rho[v_i * v_j]. (Char. size: 0.0052409470892993285)
Model residual: 0.00013286133667685254


In [14]:
### not fixed (obviously)
#veqconst_terms = ['1', 'v_j * v_j']
veqconst_terms = ['rho', 'rho * v_j * v_j']
col_numbers = [find_term(terms0, name) for name in veqconst_terms]
Xi, lambd = regress(Q0, col_numbers)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {terms0[i]}. (Char. size: {char_sizes0[i]})")
print("Model residual:", lambd)

[Term 16] -0.004286563115825593 * rho. (Char. size: 0.1464261231728444)
[Term 28] 1.0 * rho * v_j * v_j. (Char. size: 0.0005121399208783936)
Model residual: 0.0005697337514007149


In [ ]:
rho dt^2 v = 10^-4 dt rho * grad rho